In [1]:
!pip install keras

In [1]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
import pandas as pd
from tqdm import tqdm
import lightgbm as ltb
from imblearn.over_sampling import RandomOverSampler, SMOTE
import pickle
import joblib

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import ResNet50
from keras.applications.resnet import preprocess_input

2022-12-21 14:16:23.556768: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-21 14:16:23.838109: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-21 14:16:23.838129: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-21 14:16:24.706474: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
directory = 'all_data/train/'
directory2 = 'all_data/train2/'
directorytf = 'all_data/train'
directorytf_aug = 'all_data/augment'
directory_test = 'all_data/test/'
pic_x = []
pic_xname = []

In [4]:
pic_xs = {}
pic_xhsv = {}
pic_xs_test = {}

In [17]:
from matplotlib import image, pyplot
import cv2 as cv
import sys

In [18]:
def append_value(dict_obj, key, value):
    # Check if key exist in dict or not
    if key in dict_obj:
        # Key exist in dict.
        # Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            # If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        # Append the value in list
        dict_obj[key].append(value)
    else:
        # As key is not in dict,
        # so, add key-value pair
        dict_obj[key] = value

In [ ]:
# masks of lab format   a - red & green,  b- yellow and blue
for filedir in os.listdir(directory):
    f = os.path.join(directory, filedir)
    for filename in os.listdir(f):
        #print(directory+filedir+'/'+filename)
        pic = cv.imread(directory+filedir+'/'+filename)  # mb dir with linux command
        pic1 = cv.resize(pic, (100,100))
        pic2 = cv.cvtColor(pic1, cv.COLOR_BGR2LAB)
        a_channel = pic2[:,:,1]
        th = cv.threshold(a_channel,125,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)[1]
        append_value(pic_xs, filedir, th)
        #pic_y.append(filedir) # maybe hstack or vstack

In [14]:
# no-back of lab format   a - red & green,  b- yellow and blue
for filedir in os.listdir(directory):
    f = os.path.join(directory, filedir)
    for filename in os.listdir(f):
        #print(directory+filedir+'/'+filename)
        pic = cv.imread(directory+filedir+'/'+filename)  # mb dir with linux command
        pic1 = cv.resize(pic, (100,100))
        pic2 = cv.cvtColor(pic1, cv.COLOR_BGR2LAB)
        a_channel = pic2[:,:,1]
        th = cv.threshold(a_channel,125,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)[1]
        masked = cv.bitwise_and(pic2,  pic2, mask = th)
        append_value(pic_xhsv, filedir, masked)
        #pic_y.append(filedir) # maybe hstack or vstack

error: OpenCV(4.6.0) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [7]:
def get_img_data(directory, dct, size=(100,100), dir_type=''):
    for filedir in os.listdir(directory):
        f = os.path.join(directory, filedir)
        for filename in os.listdir(f+'/'+dir_type):
            #print(directory+filedir+'/'+filename)
            if dir_type=='':
                pic = cv.imread(directory+filedir+'/'+filename)
            else:
                pic = cv.imread(directory+filedir+'/'+dir_type+'/'+filename)  # mb dir with linux command
            pic1 = cv.resize(pic, (100,100))
            pic2 = cv.cvtColor(pic1, cv.COLOR_BGR2LAB)
            a_channel = pic2[:,:,1]
            th = cv.threshold(a_channel,125,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)[1]
            masked = cv.bitwise_and(pic2,  pic2, mask = th)
            append_value(dct, filedir, masked)
            #pic_y.append(filedir) # maybe hstack or vstack

In [7]:
# no-back of RGB format   a - red & green,  b- yellow and blue #augmented
for filedir in os.listdir(directory):
    f = os.path.join(directory, filedir)
    for filename in os.listdir(f+'/aug'):
        #print(directory+filedir+'/'+filename)
        pic = cv.imread(directory+filedir+'/'+'aug/'+filename)  # mb dir with linux command
        pic1 = cv.resize(pic, (100,100))
        pic2 = cv.cvtColor(pic1, cv.COLOR_BGR2LAB)
        a_channel = pic2[:,:,1]
        th = cv.threshold(a_channel,125,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)[1]
        masked_hsv = cv.bitwise_and(pic2,  pic2, mask = th)
        masked = cv.cvtColor(masked_hsv, cv.COLOR_HSV2RGB)
        append_value(pic_xs, filedir, masked)
        #pic_y.append(filedir) # maybe hstack or vstack

In [ ]:
# no-back of RGB format   a - red & green,  b- yellow and blue #augmented fixed
for filedir in os.listdir(directory):
    f = os.path.join(directory, filedir)
    for filename in os.listdir(f+'/aug'):
        #print(directory+filedir+'/'+filename)
        pic = cv.imread(directory+filedir+'/'+'aug/'+filename)  # mb dir with linux command
        pic1 = cv.resize(pic, (100,100))
        pic2 = cv.cvtColor(pic1, cv.COLOR_BGR2RGB)
        #a_channel = pic2[:,:,1]
        #th = cv.threshold(a_channel,125,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)[1]
        #masked_hsv = cv.bitwise_and(pic2,  pic2, mask = th)
        #masked = cv.cvtColor(masked_hsv, cv.COLOR_HSV2RGB)
        append_value(pic_xs, filedir, pic2)
        #pic_y.append(filedir) # maybe hstack or vstack

In [8]:
get_img_data(directory, pic_xs, dir_type='aug')

In [20]:
len(pic_xs['Charlock'])

700

In [8]:
grasses = [i for i in pic_xs.keys()]

In [9]:
grasses

['Loose Silky-bent',
 'Charlock',
 'Small-flowered Cranesbill',
 'Cleavers',
 'Sugar beet',
 'Maize',
 'Common wheat',
 'Black-grass',
 'Shepherds Purse',
 'Fat Hen',
 'Scentless Mayweed',
 'Common Chickweed']

In [10]:
data_lst = []
for i in grasses:
  for j in (pic_xs[i]):
    data_lst.append(i)

In [11]:
x_lst = []
for i in grasses:
  for j in (pic_xs[i]):
    #for k in j:
    x_lst.append(j)

In [26]:
X_raw = np.reshape(np.array(x_lst)/255, (8400, 100*100*3))

In [12]:
X = np.array(x_lst)
X = preprocess_input(X)

In [13]:
X.shape

(8400, 100, 100, 3)

In [14]:
df = pd.DataFrame(data_lst, columns=['Y'])

In [15]:
le = LabelEncoder().fit(df.Y)
df['Yl'] = le.transform(df.Y)

In [16]:
df['Yl'].unique()

array([ 6,  1, 10,  2, 11,  7,  4,  0,  9,  5,  8,  3])

In [17]:
arr = df.Yl.to_numpy()

In [18]:
Y = np.reshape(arr, (-1,))

In [7]:
seed = 0

# Dimentionality reduction

In [24]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(svd_solver = 'arpack')
pca.fit(X)

PCA(svd_solver='arpack')

In [ ]:
pipe = Pipeline([('pca', PCA(svd_solver = 'arpack')), 
                  ('ltb', SVC(random_state=seed, C=100, kernel='rbf', degree=1))])
scores = cross_val_score(pipe, X_tf, Y, scoring='f1_micro', cv=4)

In [ ]:
pipe = Pipeline([('pca', PCA(svd_solver = 'arpack')), 
                  ('svc', ltb.LGBMClassifier(random_state=seed, max_depth=10, objective='multiclass', min_child_samples=2,
                              learning_rate=0.1, n_estimators=20, n_jobs=-1, class_weight='balanced'))])
scores = cross_val_score(pipe, X, Y, scoring='f1_micro', cv=4)

In [ ]:
scores, np.mean(scores)

In [ ]:
X_new = pca.transform(X)

# Balancing data

In [5]:
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 4.3 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 11.3 MB/s eta 0:00:00
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 10.3 MB/s eta 0:00:0000:0100:01
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 10.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 10.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 10.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 10.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.1 MB/s eta 0:00:00a 0:00:01
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from keras.preprocessing.image import ImageDataGenerator

2022-11-24 10:39:03.066300: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-24 10:39:03.250777: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/viktor/Projects/Kaggle_Projs/Plants_CV/venv/lib/python3.10/site-packages/cv2/../../lib64:
2022-11-24 10:39:03.250812: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-24 10:39:03.979236: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic li

In [31]:
!pip install Augmentor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.2/829.2 kB 1.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=cc63e191e96738cedb4f7005ef9c4a4fc3a1dc1a891fa12fd1a6b1a082c48317
  Stored in directory: /home/viktor/.cache/pip/wheels/dc/16/09/eb08b4e34e6b638f113d2018cf0b22de1d8dca22a3a71873f7
Successfully built future


In [32]:
import Augmentor

In [42]:
for fold in grasses:
    os.mkdir(directorytf_aug+fold)

In [50]:
for fold in grasses:
    p = Augmentor.Pipeline(source_directory=directory+fold, output_directory='aug')
    p.rotate(probability=0.5, max_left_rotation=25, max_right_rotation=25)
    p.rotate_random_90(0.5)
    p.sample(700)
print('All classes are processed')

Initialised with 654 image(s) found.
Output directory set to all_data/train/Loose Silky-bent/aug.

Processing <PIL.PngImagePlugin.PngImageFile image mode=RGB size=176x176 at 0x7FEF08AD70D0>: 100%|██████████████████████████████████████████████████████████████████████████████████████| 700/700 [00:13<00:00, 50.72 Samples/s]


Initialised with 390 image(s) found.
Output directory set to all_data/train/Charlock/aug.

Processing <PIL.Image.Image image mode=RGB size=244x244 at 0x7FEF08A79D80>: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 700/700 [00:08<00:00, 87.23 Samples/s]


Initialised with 496 image(s) found.
Output directory set to all_data/train/Small-flowered Cranesbill/aug.

...

Initialised with 516 image(s) found.
Output directory set to all_data/train/Scentless Mayweed/aug.

Processing <PIL.Image.Image image mode=RGB size=138x138 at 0x7FEEC8C9A140>: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 700/700 [00:03<00:00, 218.98 Samples/s]


Initialised with 611 image(s) found.
Output directory set to all_data/train/Common Chickweed/aug.

Processing <PIL.Image.Image image mode=RGB size=156x156 at 0x7FEEC8A01840>: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 700/700 [00:02<00:00, 245.44 Samples/s]

All classes are processed


In [18]:
ros = RandomOverSampler(random_state=seed)
X_bal, Y_bal = ros.fit_resample(X, Y)

In [21]:
smo = SMOTE(random_state=seed)
X_bal, Y_bal = smo.fit_resample(X, Y)

In [5]:
datagen = ImageDataGenerator(
    rotation_range=45,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='constant'
)

In [44]:
aug_dir = 'all_data/'
directory = 'all_data/train/'
directorytf = 'all_data/train'
directorytf_aug = 'all_data/augment/'
pic_x = []
pic_y = []

In [29]:
for
    for i, batch in enumerate(datagen.flow_from_directory(directory=directorytf,
                                             batch_size=16,
                                             target_size=(100, 100),
                                             color_mode='rgb',
                                             class_mode='categorical',
                                             save_to_dir=directorytf_aug,
                                             save_prefix='aug',
                                             save_format='png',
                                            seed=seed)):
      if i > 650:
        break


Found 4750 images belonging to 12 classes.


In [26]:
for fold in grasses:
    #os.mkdir(directorytf_aug+fold)
    i = 0
    for batch in datagen.flow_from_directory(directory=(directory+fold),
                                             batch_size=32,
                                             target_size=(100, 100),
                                             color_mode='rgb',
                                             #class_mode='binary',
                                             save_to_dir=directorytf_aug+fold,
                                             save_prefix='aug',
                                             save_format='png'):
      i+=1
      if i > 650:
        break


Found 4750 images belonging to 12 classes.


# Test preparation

In [ ]:
#test dataset
for filename in os.listdir(directory_test):
    #print(filename)
    pic = cv.imread(directory_test+filename)  # mb dir with linux command
    pic1 = cv.resize(pic, (100,100))
    pic2 = cv.cvtColor(pic1, cv.COLOR_BGR2LAB)
    a_channel = pic2[:,:,1]
    th = cv.threshold(a_channel,125,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)[1]
    masked_hsv = cv.bitwise_and(pic2,  pic2, mask = th)
    masked = cv.cvtColor(masked_hsv, cv.COLOR_HSV2RGB)
    pic_x.append(masked)
    pic_xname.append(filename)

In [ ]:
#test dataset unmasked
for filename in os.listdir(directory_test):
    #print(filename)
    pic = cv.imread(directory_test+filename)  # mb dir with linux command
    pic1 = cv.resize(pic, (100,100))
    pic2 = cv.cvtColor(pic1, cv.COLOR_BGR2RGB)
    #a_channel = pic2[:,:,1]
    #th = cv.threshold(a_channel,125,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)[1]
    #masked_hsv = cv.bitwise_and(pic2,  pic2, mask = th)
    #masked = cv.cvtColor(masked_hsv, cv.COLOR_HSV2RGB)
    pic_x.append(pic2)
    pic_xname.append(filename)

In [ ]:
X = np.array(pic_x)
X = preprocess_input(X)

# Feature extraction with ResNet50

In [21]:
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(100,100,3))

2022-12-21 15:46:55.379496: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-21 15:46:55.380772: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (viktor-furda): /proc/driver/nvidia/version does not exist
2022-12-21 15:46:55.389825: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 106, 106, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 50, 50, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 50, 50, 64)   256         ['conv1_conv[0][0]']      

 conv2_block3_2_conv (Conv2D)   (None, 25, 25, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 25, 25, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 25, 25, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_3_conv (Conv2D)   (None, 25, 25, 256)  16640       ['conv2_block3_2_relu[0][0]']    
                                                                                                  
 conv2_blo

 conv3_block3_2_conv (Conv2D)   (None, 13, 13, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 13, 13, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 13, 13, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_3_conv (Conv2D)   (None, 13, 13, 512)  66048       ['conv3_block3_2_relu[0][0]']    
                                                                                                  
 conv3_blo

 conv4_block2_2_conv (Conv2D)   (None, 7, 7, 256)    590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 7, 7, 256)   1024        ['conv4_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_2_relu (Activatio  (None, 7, 7, 256)   0           ['conv4_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_3_conv (Conv2D)   (None, 7, 7, 1024)   263168      ['conv4_block2_2_relu[0][0]']    
                                                                                                  
 conv4_blo

 conv4_block5_2_relu (Activatio  (None, 7, 7, 256)   0           ['conv4_block5_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_3_conv (Conv2D)   (None, 7, 7, 1024)   263168      ['conv4_block5_2_relu[0][0]']    
                                                                                                  
 conv4_block5_3_bn (BatchNormal  (None, 7, 7, 1024)  4096        ['conv4_block5_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_add (Add)         (None, 7, 7, 1024)   0           ['conv4_block4_out[0][0]',       
                                                                  'conv4_block5_3_bn[0][0]']      
          

 conv5_block2_2_relu (Activatio  (None, 4, 4, 512)   0           ['conv5_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block2_3_conv (Conv2D)   (None, 4, 4, 2048)   1050624     ['conv5_block2_2_relu[0][0]']    
                                                                                                  
 conv5_block2_3_bn (BatchNormal  (None, 4, 4, 2048)  8192        ['conv5_block2_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_add (Add)         (None, 4, 4, 2048)   0           ['conv5_block1_out[0][0]',       
                                                                  'conv5_block2_3_bn[0][0]']      
          

In [23]:
new_X = base_model.predict(X, batch_size=16)

50/50 [==============================] - 8s 140ms/step


In [24]:
new_X.shape

(794, 4, 4, 2048)

In [25]:
#test
X_test = np.reshape(new_X, (794, 4*4*2048))

In [27]:
#unbalanced
X_tf = np.reshape(new_X, (4750, 4*4*2048))

In [24]:
# augmented
X_tf = np.reshape(new_X, (8400, 4*4*2048))

In [25]:
X_tf.shape

(8400, 32768)

In [28]:
X_conc = np.hstack((X_tf, X_raw))

In [29]:
X_conc.shape

(8400, 62768)

In [14]:
X_test.shape

(794, 32768)

In [30]:
X_rew = np.reshape(np.array(pic_x)/255, (794, 100*100*3))

In [31]:
X_testf = np.hstack((X_test, X_rew))

In [32]:
X_testf.shape

(794, 62768)

# Array saving

array([6, 6, 6, ..., 3, 3, 3])

In [33]:
np.save('x_conc.pkl', X_conc)
np.save('y.pkl', Y)

In [4]:
X_conc = np.load('x_conc.pkl.npy')
Y = np.load('y.pkl.npy')

In [5]:
X_conc.shape, Y.shape

((8400, 62768), (8400,))

# Model Training

In [8]:
clf = ltb.LGBMClassifier(random_state=seed, max_depth=10, objective='multiclass',
                         min_child_samples=2, learning_rate=0.1, n_estimators=100, n_jobs=-1)

In [28]:
clf.fit(X_tf, Y)

LGBMClassifier(max_depth=10, min_child_samples=2, objective='multiclass',
               random_state=0)

In [9]:
clf.fit(X_conc, Y)

LGBMClassifier(max_depth=10, min_child_samples=2, objective='multiclass',
               random_state=0)

In [10]:
joblib.dump(clf, 'gbm_aug3.pkl')

['gbm_aug3.pkl']

In [30]:
joblib.dump(le, 'le.pkl')

['le.pkl']

# Test dataset tricks

In [15]:
clf = joblib.load('gbm_aug.pkl')

In [11]:
le = joblib.load('le.pkl')

In [33]:
labels = clf.predict(X_testf)

In [34]:
Y = le.inverse_transform(labels)

In [35]:
Y

array(['Common wheat', 'Small-flowered Cranesbill', 'Fat Hen',
       'Common Chickweed', 'Sugar beet', 'Fat Hen', 'Cleavers',
       'Black-grass', 'Scentless Mayweed', 'Small-flowered Cranesbill',
       'Loose Silky-bent', 'Small-flowered Cranesbill',
       'Common Chickweed', 'Maize', 'Black-grass', 'Fat Hen',
       'Black-grass', 'Common Chickweed', 'Loose Silky-bent', 'Maize',
       'Fat Hen', 'Maize', 'Small-flowered Cranesbill', 'Charlock',
       'Scentless Mayweed', 'Fat Hen', 'Shepherds Purse',
       'Loose Silky-bent', 'Common Chickweed', 'Cleavers', 'Maize',
       'Maize', 'Loose Silky-bent', 'Small-flowered Cranesbill',
       'Common Chickweed', 'Loose Silky-bent', 'Common Chickweed',
       'Fat Hen', 'Charlock', 'Scentless Mayweed', 'Charlock',
       'Common Chickweed', 'Shepherds Purse', 'Charlock', 'Charlock',
       'Cleavers', 'Maize', 'Charlock', 'Common Chickweed', 'Sugar beet',
       'Common Chickweed', 'Fat Hen', 'Loose Silky-bent', 'Fat Hen',
       'Ch

In [36]:
df = pd.DataFrame({'file': pic_xname,
                  'species': Y})

In [38]:
df.to_csv('test_result3.csv', index=False)

# Model Testing

In [38]:
clf = SVC(random_state=seed, C=100, kernel='rbf', degree=1)
scores = cross_val_score(clf, X_tf, Y, scoring='f1_micro', cv=4)

In [ ]:
clf = GradientBoostingClassifier(random_state=seed, max_depth=5, min_samples_split = 10, min_samples_leaf=2, learning_rate=0.1, n_estimators=5)
scores = cross_val_score(clf, X, Y, scoring='f1_micro', cv=4)

In [ ]:
!pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
import lightgbm as ltb

In [27]:
#lgbm
clf = ltb.LGBMClassifier(random_state=seed, max_depth=10, objective='multiclass', min_child_samples=2, learning_rate=0.1, n_estimators=100, n_jobs=-1)
scores2 = cross_val_score(clf, X_tf, Y, scoring='f1_micro', cv=4)

In [ ]:
#MLP
clf = MLPClassifier(random_state=seed, hidden_layer_sizes=(512, 256, 128, 64, 32, 16, 4), max_iter=500)
scores2 = cross_val_score(clf, X, Y, scoring='f1_micro', cv=4)

In [21]:
clf.fit(X, Y)

LGBMClassifier(max_depth=10, min_child_samples=2, n_estimators=20,
               objective='multiclass', random_state=0)

In [39]:
scores, np.mean(scores)

(array([0.63761905, 0.61857143, 0.61333333, 0.59809524]), 0.6169047619047618)

In [28]:
scores2, np.mean(scores2)

(array([0.81333333, 0.79857143, 0.81142857, 0.79047619]), 0.8034523809523809)

In [ ]:
regl.fit(X, Y)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  9.0min finished


LogisticRegression(C=0.01, max_iter=1000, n_jobs=-1, random_state=0,
                   verbose=True)